# Clustering Crypto

In [71]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [72]:
# Load the crypto_data.csv dataset.
file_path= r"C:\Users\jbrad\OneDrive\Desktop\Bootcamp_Classwork\Cryptocurrencies\Resources\crypto_data.csv"
crypto_data_df = pd.read_csv(file_path)
crypto_data_df.head() 

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [73]:
# Keep all the cryptocurrencies that are being traded.
trading_crypto_df = crypto_data_df[crypto_data_df['IsTrading'] == True]
trading_crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [74]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_alg_df = trading_crypto_df.dropna(subset=['Algorithm'])
crypto_alg_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [75]:
# Remove the "IsTrading" column. 
cleaning_crypto_df = crypto_alg_df.drop(['IsTrading'], axis=1)
cleaning_crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [76]:
# Remove rows that have at least 1 null value.
nonull_crypto_df = cleaning_crypto_df.dropna(how='any', axis=0)
nonull_crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [77]:
# Keep the rows where coins are mined.
mined_crypto_df = nonull_crypto_df[nonull_crypto_df['TotalCoinsMined'] >0]
mined_crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [78]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = mined_crypto_df.drop(['Unnamed: 0' , 'Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'], axis=1)
crypto_names_df.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [79]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
noname_crypto_df = mined_crypto_df.drop(['CoinName', 'Unnamed: 0'], axis=1)
noname_crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [80]:
# Use get_dummies() to create variables for text features.
new = pd.get_dummies(noname_crypto_df, columns=['Algorithm', 'ProofType'])
new.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# Standardize the data with StandardScaler().
new_crypto_scaled = StandardScaler().fit_transform(new)
print(new_crypto_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [82]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
new_pca = pca.fit_transform(new_crypto_scaled)
new_pca

array([[-0.33091416,  0.97481525, -0.53761078],
       [-0.31425403,  0.97489493, -0.53792421],
       [ 2.30374118,  1.57483708, -0.52295285],
       ...,
       [ 0.31639199, -2.25800718,  0.38743555],
       [-0.13502194, -1.94353228,  0.47478575],
       [-0.28598129,  0.80954964, -0.32079079]])

In [83]:
# Create a DataFrame with the three principal components.
new_pca_df = pd.DataFrame(data=new_pca, columns=['PC 1', 'PC 2', 'PC 3'], index = noname_crypto_df.index)
new_pca_df.head()

,PC 1,PC 2,PC 3
0,-0.330914,0.974815,-0.537611
2,-0.314254,0.974895,-0.537924
5,2.303741,1.574837,-0.522953
7,-0.145798,-1.299020,0.186292
8,-0.150199,-2.004917,0.356090


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [84]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(new_pca_df)
    inertia.append(km.inertia_)
    
elbow_data = {'k': k, 'inertia': inertia}
crypto_elbow_df = pd.DataFrame(elbow_data)
crypto_elbow_df.hvplot.line(x='k', y='inertia', title='Crypto Elbow Curve', xticks=k)


C:\Users\jbrad\anaconda3\envs\PythonData\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [85]:
# Initialize the K-Means model.
crypto_model = KMeans(n_clusters=4, random_state=0)

# Fit the model
crypto_model.fit(new_pca_df)

# Predict clusters
predictions = crypto_model.predict(new_pca_df)

predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [86]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
crypto_clustered_df = pd.concat([noname_crypto_df, new_pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
crypto_clustered_df['CoinName'] = crypto_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
crypto_clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(crypto_clustered_df.shape)
crypto_clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.330914,0.974815,-0.537611,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314254,0.974895,-0.537924,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.303741,1.574837,-0.522953,EliteCoin,0
7,SHA-256,PoW,1.792718e+07,21000000,-0.145798,-1.299020,0.186292,Bitcoin,3
8,Ethash,PoW,1.076842e+08,0,-0.150199,-2.004917,0.356090,Ethereum,3
9,Scrypt,PoW,6.303924e+07,84000000,-0.161889,-1.148667,-0.035534,Litecoin,3
10,X11,PoW/PoS,9.031294e+06,22000000,-0.399639,1.206023,-0.407103,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.146992,-2.190521,0.361217,Monero,3
12,Ethash,PoW,1.133597e+08,210000000,-0.148642,-2.005024,0.356086,Ethereum Classic,3
13,Equihash,PoW,7.383056e+06,21000000,-0.135021,-1.943532,0.474786,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [87]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_clustered_df,
    x='PC 1',
    y='PC 2',
    z='PC 3',
    color='Class',
    symbol='Class',
    width=800,
    hover_name='CoinName',
    hover_data=['Algorithm'])

fig.update_layout(legend=dict(x=0, y=1))
fig.show()
                 



In [88]:
# Create a table with tradable cryptocurrencies.
crypto_clustered_df.hvplot.table(sortable=True, selectable=True)

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class]

In [89]:
# Print the total number of tradable cryptocurrencies.
index = crypto_clustered_df.index
len(index)
print("There are " + str(len(index)) + " " + "tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [90]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mms = MinMaxScaler()
crypto_scaled = mms.fit_transform(crypto_clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
crypto_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [91]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
data=crypto_scaled, columns= ['TotalCoinSupply', 'TotalCoinsMined'], index = crypto_clustered_df.index)


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName']= crypto_clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = crypto_clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000,42 Coin,0
2,5.320000e-04,0.001066,404Coin,0
5,3.141593e-01,0.029576,EliteCoin,0
7,2.100000e-05,0.000018,Bitcoin,3
8,0.000000e+00,0.000109,Ethereum,3
9,8.400000e-05,0.000064,Litecoin,3
10,2.200000e-05,0.000009,Dash,0
11,0.000000e+00,0.000017,Monero,3
12,2.100000e-04,0.000115,Ethereum Classic,3
13,2.100000e-05,0.000007,ZCash,3


In [92]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x='TotalCoinsMined', y='TotalCoinSupply', by='Class', hover_cols=['CoinName'])



:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)